# APLICAÇÃO DO ALGORTIMO DE XGBOOST

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

Leitura do dataset final.

In [2]:
dataset = pd.read_csv('DATASET_FINAL_PARA_APLICAÇÃO_DOS_ALGORITIMOS.csv')
dataset.head()

,target,High,Low,Open,Close,Volume,Adj Close,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa AC,...,Empréstimos e Financiamentos PNC,Outros PNC,Tributos Diferidos PNC,Provisões PNC,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Lucros,Lucros/Prejuízos Acumulados
0,Bad,42.900002,19.440001,19.985001,22.750000,3.449634e+07,12.533106,2885705.0,1715015.0,374622.0,...,471957.0,17108.0,81381.776590,119916.0,1.0,1033561.0,415955.0,147287.0,2.581270e+05,234891.000000
1,Bad,15.240000,9.320000,13.980000,11.150000,4.231002e+07,11.624119,63253422.0,20793485.0,3111608.0,...,18649526.0,906897.0,944546.000000,2946308.0,19644613.0,11596044.0,6881998.0,-69861.0,4.148446e+06,198944.931086
2,Bad,27.309999,21.730000,24.750000,24.600000,1.827805e+07,23.324638,4910968.0,882855.0,238905.0,...,1739126.0,4326.0,70845.000000,14419.0,6617.0,2487188.0,1231313.0,473382.0,7.824930e+05,109693.674100
3,Bad,30.350000,26.270000,29.610001,27.139999,3.827353e+07,25.771338,3454612.0,1126358.0,191491.0,...,564759.0,67259.0,371338.000000,28151.0,1.0,1600873.0,1413608.0,20077.0,7.068100e+04,96429.000000
4,Bad,22.500000,11.583700,19.866100,14.940000,1.157091e+07,16.613333,8265026.0,3633042.0,123715.0,...,2646948.0,105024.0,16875.367971,320365.0,108.0,2861085.0,3632101.0,24773.0,5.024450e+05,-795460.000000


Só mostrando que o alvo a ser predito está balanceado.

In [3]:
dataset['target'].value_counts()

Good    657
Bad     657
Name: target, dtype: int64

Aqui normalizamos o dataset, trazendo para uma escala de 0 a 1, ainda que isso não faça muita diferença em árvore de decisão.

In [4]:
normalizador = MinMaxScaler(feature_range=(0,1))

normalizador_features = normalizador.fit_transform(dataset.drop('target',axis=1))

dataset_normalizador = pd.DataFrame(normalizador_features,columns=dataset.columns[1:])

X_train, X_test, y_train, y_test = train_test_split(normalizador_features, dataset['target'], test_size=0.2)

In [5]:
X_train

array([[0.1052172 , 0.05924601, 0.11354157, ..., 0.04743568, 0.03025748,
        0.46372975],
       [0.12649653, 0.09791676, 0.09304432, ..., 0.03625699, 0.85510228,
        0.59424387],
       [0.10281955, 0.10192647, 0.10344045, ..., 0.04619003, 0.00884957,
        0.47774122],
       ...,
       [0.22725986, 0.22581451, 0.22136001, ..., 0.04950551, 0.02649368,
        0.4842152 ],
       [0.06086012, 0.05740217, 0.06323959, ..., 0.04770481, 0.00567275,
        0.46951981],
       [0.21426283, 0.11993298, 0.12861089, ..., 0.04749007, 0.00150068,
        0.45785942]])

Aqui calculamos o tempos do processo de Grid Seach onde ele busca os melhores parâmetros que especulamos para cada hiperparâmetro.

Assim, printamos os melhores valores estimados.

In [6]:
%%time

xgb = XGBClassifier()

parameter = {'learning_rate': [0.01, 0.05, 0.001],      #so called `eta` value
             'max_depth': [10,15,25],
             'min_child_weight': [1, 5, 11],
             'subsample': [0.1, 0.5, 0.8, 1.0],
             'colsample_bytree': [0.1, 0.5, 0.7],
             'n_estimators': [125, 150,175],           #number of trees, change it to 1000 for better results
              'missing':[-999]}



xgb_grid = GridSearchCV(xgb,
                        parameter,
                        cv = 10,
                        refit='accuracy')

xgb_grid.fit(X_train, y_train)
xgb_grid.best_estimator_

/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


CPU times: user 1h 14min 22s, sys: 1min 24s, total: 1h 15min 46s
Wall time: 1h 16min 9s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=-999, n_estimators=175, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

Aqui, extraimos as predições feitas nos dados de teste.

In [7]:
predictions = xgb_grid.best_estimator_.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         Bad       0.83      0.80      0.81       128
        Good       0.81      0.84      0.83       135

    accuracy                           0.82       263
   macro avg       0.82      0.82      0.82       263
weighted avg       0.82      0.82      0.82       263



## CLASSIFICAÇÃO. 

Classification report nos traz as principais métricas de desempenho do algoritmo nesses dados.

In [8]:
xgb_grid.best_score_

0.8306374881065651

E ainda temos o desempenho com base no score do grid search com o cross validation que foi "acurácia".

In [9]:
import pickle
from joblib import dump, load


s = pickle.dumps(xgb_grid)
dump(xgb_grid, 'XGBoost.joblib')

['XGBoost.joblib']

In [10]:
xgb_grid_cv = load('XGBoost.joblib')

xgb_grid_cv.best_estimator_


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=-999, n_estimators=175, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)